# Data collection

In [ ]:
! wget https://raw.githubusercontent.com/FakeNewsChallenge/fnc-1/master/train_stances.csv
! wget https://raw.githubusercontent.com/FakeNewsChallenge/fnc-1/master/train_bodies.csv
! wget https://raw.githubusercontent.com/FakeNewsChallenge/fnc-1/master/competition_test_stances.csv
! wget https://raw.githubusercontent.com/FakeNewsChallenge/fnc-1/master/competition_test_bodies.csv

# Dependencies

In [ ]:
! pip install pandas transformers datasets scikit-learn simpletransformers torch

# Reading data

In [3]:
import pandas as pd

train_stances = pd.read_csv('train_stances.csv', header=None)
train_bodies = pd.read_csv('train_bodies.csv', header=None)
test_stances = pd.read_csv('competition_test_stances.csv', header=None)
test_bodies = pd.read_csv('competition_test_bodies.csv', header=None)

In [6]:
from sklearn.model_selection import train_test_split

def data_extract(stances, bodies):
    map = {'agree': 0, 'disagree':1, 'discuss':2, 'unrelated':3}
    headline = []
    body = []
    stance = []
    
    for i in range(1,len(stances)):
        match = bodies[bodies[0]==stances[1][i]][:]
        headline.append(stances[0][i])
        body.append(match[1].to_string())
        stance.append(map[stances[2][i]])
        
    return headline, body, stance
   
headline, body, stance = data_extract(train_stances, train_bodies)
data = list(zip(headline, body, stance))

df = pd.DataFrame(data, columns=['text_a', 'text_b', 'labels'])
train_df, val_df = train_test_split(df)
labels_val = pd.Series(val_df['labels']).to_numpy()

headline, body, stance = data_extract(test_stances, test_bodies)
data = list(zip(headline, body, stance))

test_df = pd.DataFrame(data, columns=['text_a', 'text_b', 'labels'])
labels_test = pd.Series(test_df['labels']).to_numpy()


In [12]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from simpletransformers.classification import ClassificationModel
model = ClassificationModel('bert', 'bert-base-uncased', use_cuda=False,num_labels=4, args={
    'learning_rate':3e-5,
    'num_train_epochs': 1,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'process_count': 10,
    'train_batch_size': 4,
    'eval_batch_size': 4,
    'max_seq_length': 64,
    'fp16': True
})

model.train_model(train_df)

ModuleNotFoundError: No module named '_lzma'

In [26]:
print(train_df["label"])

43793    3
13931    3
10528    3
17478    3
15801    2
        ..
12786    3
6503     2
21127    3
13154    3
41529    3
Name: label, Length: 37479, dtype: int64
